# Processing Earthquake Events Data with `obspy` and `csp`

## Introduction

In this example, we will use CSP to process earthquake events and plot them in a map using a [Perspective](https://perspective.finos.org) widget.

First, we need to install a few extra libraries:
- `obspy`, for reading the earthquake stream from USGS;
- `perspective-python`, to create the visualization of the data; and 
- `cartopy`, for plotting the individual events in the USGS catalog (this is optional).

**Note:** This example has been tested for `jupyterlab==4.2.0` and `perspective-python==2.10.0`.

You can install these dependencies in your Python environment with the following command:

```
pip install obspy cartopy jupyterlab==4.2.0 perspective-python==2.10.0
```

#### Reading realtime data from USGS as QuakeML

The [USGS website](https://earthquake.usgs.gov/earthquakes) provides several feeds with recent seismic events, and we will read the "All day" feed containing all seismic events of the past 24h. Using `obspy`, we can read the feed in the [QUAKEML](https://earthquake.usgs.gov/earthquakes/feed/v1.0/quakeml.php) format as a `catalog`:

In [ ]:
from obspy import read_events

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.quakeml"
catalog = read_events(url, format="QUAKEML")

In [ ]:
catalog

This catalog's events are not ordered, but we can sort them easily:

In [ ]:
def sorting(event):
    return event.origins[0].time
catalog.events.sort(key=sorting)

Now, we can calso use [cartopy](https://scitools.org.uk/cartopy/docs/latest/) to to plot all events in a world map:

In [ ]:
catalog.plot();

To see what these event objects are, we can look closer:

In [ ]:
event = catalog[0]
event

This feed lists different kinds of events, noted by `event.event_type`. We can also inspect location, time and magnitude information:

In [ ]:
event.origins, event.magnitudes

This feed is updated every minute, meaning we get a historical dataset of all seismic events in the past 24h, but we also get a continually updated feed that adds new events as they are entered into this feed (all events will contain this `creation_time` information).

## With CSP

We can now use CSP to read the same data in either realtime or simulation modes, by building a `PushPullAdapter`. This adapter combines a realtime or `PushAdapter` and a historical or `PullAdapter` into a single implementation. This makes it easy to switch from historical mode to realtime mode at runtime. 

Because we want to visualize the results, we will start by creating a Perspective widget containing a table and a world map. This widget will be updated every time a new event is detected (and the corresponding CSP edge is *ticked*). We will read historical events from the past 6h, and run the engine for 10 minutes while we wait for new events to be added to the catalog in realtime.

In [ ]:
from perspective import PerspectiveWidget, Plugin
import ipywidgets as widgets
from datetime import datetime

# Data schema for Perspective widget
data = {
    "longitude": float,
    "latitude": float,
    "magnitude": float,
    "raw_time": datetime,
    "raw_creation_time": datetime,
    "time": str,
    "creation_time": str,
}

datagrid = PerspectiveWidget(
    data,
    plugin=Plugin.GRID,
    group_by=["raw_time"],
    columns=["longitude", "latitude", "magnitude", "raw_creation_time"],
    aggregates={
        "longitude": "last",
        "latitude": "last",
        "magnitude": "last",
        "raw_creation_time": "last",     
    },
)
worldmap = PerspectiveWidget(
    data,
    plugin=Plugin.MAP_SCATTER,
    columns=["longitude", "latitude", "magnitude", "magnitude", "time", "creation_time"],
)
# Create a tab widget with some PerspectiveWidgets inside
widget = widgets.Tab()
widget.children = [datagrid, worldmap]
widget.titles = ["All events", "World map"]
widget

In [ ]:
# PushPullAdapter
import threading
import csp
from obspy import read_events
from datetime import datetime, timedelta, timezone
from csp.impl.pushpulladapter import PushPullInputAdapter
from csp.impl.wiring import py_pushpull_adapter_def
from obspy.core.utcdatetime import UTCDateTime
import time


class EventData(csp.Struct):
    time: UTCDateTime
    longitude: float
    latitude: float
    magnitude: float
    creation_time: datetime

# Create a runtime implementation of the adapter
class FetchEventDataAdapter(PushPullInputAdapter):
    def __init__(self, interval, url):
        self._interval = interval
        self._thread = None
        self._running = False
        self._url = url

    def start(self, starttime, endtime):
        print("FetchEventDataAdapter::start")
        self._running = True
        self._thread = threading.Thread(target=self._run)
        self._thread.start()
        self._starttime = starttime
        self._endtime = endtime

    def stop(self):
        print("FetchEventDataAdapter::stop")
        if self._running:
            self._running = False
            self._thread.join()

    def _run(self):
        history = []
        live = False
        while self._running:
            now = datetime.utcnow()
            print("-------------------------------------------------------------------")
            print(f"{now}: Refreshing earthquake feed, live={live}")
            print("-------------------------------------------------------------------")
            catalog = read_events(self._url, format="QUAKEML")
            def sorting(event):
                return event.origins[0].time
            catalog.events.sort(key=sorting)
            for event in catalog:
                # tick whenever new (historical or realtime) events appear
                if event.origins[0].time.datetime > self._starttime and event.origins[0].time.datetime < self._endtime:
                    if event not in history:
                        history.append(event)
                        my_event = EventData(
                            time=event.origins[0].time,
                            longitude=event.origins[0].longitude,
                            latitude=event.origins[0].latitude,
                            magnitude=event.magnitudes[0].mag,
                            creation_time=event.origins[0].creation_info.creation_time.datetime,
                        )
                        # For a PushPullAdapter, push_tick takes 3 arguments: push_tick(live: bool, time: datetime, value: csp.Struct)
                        self.push_tick(live, my_event.time.datetime, my_event)
            # After the first full pass at the events catalog, we can turn on live=True and wait for real time events
            live=True
            interval = self._interval.total_seconds()
            time.sleep(interval)

# Create the graph-time representation of our adapter
FetchEventData = py_pushpull_adapter_def("FetchEventData", FetchEventDataAdapter, csp.ts[EventData], interval=timedelta, url=str)

@csp.node
def update_widget(event: csp.ts[EventData], widget: widgets.widgets.widget_selectioncontainer.Tab):
    if csp.ticked(event):
        # widget.children = [datagrid, worldmap]
        data = {
            "raw_time": [event.time],
            "longitude": [event.longitude],
            "latitude": [event.latitude],
            "magnitude": [event.magnitude],
            "raw_creation_time": [event.creation_time],
            "time": [str(event.time)],
            "creation_time": [str(event.creation_time)],
        }
        widget.children[0].update(data)
        widget.children[1].update(data)

@csp.graph
def earthquake_graph():
    print("Start of graph building")
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.quakeml"
    interval = timedelta(seconds=60)
    earthquakes = FetchEventData(interval, url=url)
    update_widget(earthquakes, widget=widget)
    csp.add_graph_output("Earthquakes", earthquakes)
    print("End of graph building")

start = datetime.utcnow() - timedelta(hours=6)
end = datetime.utcnow() + timedelta(minutes=10)
csp.run(earthquake_graph, starttime=start, endtime=end, realtime=True)
print("Done.")

---